# Kriging of Soil Apparent Electrical Conductivity (Python)

1. Loads `Set4.2EC.csv` with coordinates already projected (Easting/Northing).
2. Subsets points by an index rule (ID % px_size == 0).
3. Estimates an empirical variogram and fits a **spherical** model.
4. Performs **Ordinary Kriging** over a regular grid.
5. Plots the predicted map in grayscale with axes and title.

**Dependencies:** NumPy, Pandas, Matplotlib, SciPy (for fitting), PyKrige (for kriging).

In [ ]:
# If needed, install packages (uncomment and run):
# %pip install numpy pandas matplotlib scipy pykrige


In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.spatial import distance
from scipy.optimize import curve_fit

try:
    from pykrige.ok import OrdinaryKriging
    PYKRIGE_AVAILABLE = True
except Exception as _e:
    PYKRIGE_AVAILABLE = False
    print('PyKrige not available. You can install it with: %pip install pykrige')


## Load data and subset
R equivalent:
```r
data.Set4.2EC <- read.csv('Set4.2EC.csv', header=TRUE)
coordinates(data.Set4.2EC) <- ~ Easting + Northing
px_size <- 10
data.Set4.2EC$ID <- 1:nrow(data.Set4.2EC)
data.Set4.2EC <- data.Set4.2EC[-which(data.Set4.2EC$ID %% px_size != 0),]
```

In [ ]:
px_size = 10  # same as R example
csv_path = 'Set4.2EC.csv'
if not os.path.exists(csv_path):
    raise FileNotFoundError('Missing Set4.2EC.csv. Place it in the working directory.')

df = pd.read_csv(csv_path)
required = {'Easting','Northing','ECto30'}
missing = required - set(df.columns)
if missing:
    raise ValueError(f'Missing required columns: {missing}')

# Create ID like in R and subset every px_size-th row
df['ID'] = np.arange(1, len(df)+1)
df_sub = df[df['ID'] % px_size == 0].copy()

X = df_sub['Easting'].to_numpy()
Y = df_sub['Northing'].to_numpy()
Z = df_sub['ECto30'].to_numpy()
len(X), df_sub.head()

## Prediction grid (bounding box + step)
R equivalent:
```r
bb <- c(N=4267868, S=4267513, E=592867, W=592082)
pred_grid <- expand.grid(Easting = seq(bb['W'], bb['E'], by=px_size),
                         Northing = seq(bb['S'], bb['N'], by=px_size))
```

In [ ]:
bb = {'N': 4267868, 'S': 4267513, 'E': 592867, 'W': 592082}
grid_x = np.arange(bb['W'], bb['E'] + px_size, px_size)
grid_y = np.arange(bb['S'], bb['N'] + px_size, px_size)
len(grid_x), len(grid_y)

## Empirical variogram and spherical model fit
We compute an empirical semi-variogram and fit a spherical model:

$\gamma(h) = nugget + sill \cdot S(h; a)$, where
$S(h; a) = 1.5 (h/a) - 0.5 (h/a)^3$ for $0 < h < a$, and $1$ for $h \ge a$.
($a$ is the range; beyond it the variogram reaches the sill + nugget.)

In [ ]:
def empirical_variogram(x, y, z, n_bins=20, max_dist=None):
    xy = np.column_stack([x, y])
    D = distance.pdist(xy)
    if max_dist is None:
        max_dist = np.percentile(D, 90)  # trim long tails
    idx = distance.squareform(np.arange(len(xy)))  # not used
    # Semivariance for pairs
    Z = z
    # Build all pairwise semivariances
    # Efficient approach: compute condensed distance matrix and pairwise semivariances
    # But scipy doesn't directly give condensed semivariances, so we compute indices
    n = len(Z)
    vals = []
    dists = []
    for i in range(n-1):
        dz = (Z[i+1:] - Z[i])
        d = distance.cdist(xy[i:i+1], xy[i+1:]).ravel()
        m = d <= max_dist
        if np.any(m):
            dists.append(d[m])
            vals.append(0.5 * dz[m]**2)
    if not dists:
        raise RuntimeError('Not enough pairs within max_dist for variogram.')
    dists = np.concatenate(dists)
    vals = np.concatenate(vals)

    bins = np.linspace(0, max_dist, n_bins+1)
    bin_centers = 0.5*(bins[:-1] + bins[1:])
    gamma = np.full(n_bins, np.nan)
    for i in range(n_bins):
        m = (dists >= bins[i]) & (dists < bins[i+1])
        if m.any():
            gamma[i] = np.mean(vals[m])
    m = np.isfinite(gamma)
    return bin_centers[m], gamma[m]

def spherical_model(h, sill, a, nugget):
    h = np.asarray(h)
    g = np.empty_like(h, dtype=float)
    # piecewise
    ha = h / a
    inside = h < a
    g[inside] = nugget + sill * (1.5*ha[inside] - 0.5*ha[inside]**3)
    g[~inside] = nugget + sill
    return g

# Estimate empirical variogram
h_emp, gamma_emp = empirical_variogram(X, Y, Z, n_bins=18)

# Fit spherical model with bounds
p0 = [np.nanmax(gamma_emp)*0.8, (np.max(h_emp)/2) or 1.0, np.nanmin(gamma_emp)*0.1]
bounds = ([1e-6, 1.0, 0.0], [np.inf, np.max(h_emp)*2, np.nanmax(gamma_emp)])
pars, cov = curve_fit(spherical_model, h_emp, gamma_emp, p0=p0, bounds=bounds, maxfev=20000)
sill_hat, range_hat, nugget_hat = pars
print('Fitted variogram params -> sill:', sill_hat, ' range:', range_hat, ' nugget:', nugget_hat)

# Plot empirical vs fitted variogram
hh = np.linspace(0, h_emp.max(), 200)
plt.figure()
plt.plot(h_emp, gamma_emp, 'o', label='Empirical')
plt.plot(hh, spherical_model(hh, *pars), '-', label='Spherical fit')
plt.xlabel('Lag distance h')
plt.ylabel('Semivariance γ(h)')
plt.title('Variogram: empirical vs spherical fit')
plt.legend()
plt.show()


## Ordinary Kriging
We use **PyKrige** with the fitted spherical parameters. If PyKrige is not available, the cell will prompt an install command.

In [ ]:
if not PYKRIGE_AVAILABLE:
    raise ModuleNotFoundError('PyKrige not installed. Install with: %pip install pykrige')

# PyKrige expects parameters [sill, range, nugget]
OK = OrdinaryKriging(
    X, Y, Z,
    variogram_model='spherical',
    variogram_parameters=[float(sill_hat), float(range_hat), float(nugget_hat)],
    enable_plotting=False,
    coordinates_type='euclidean'
)

# Execute on grid (like R's prediction grid); grid expects vectors of x and y
z_krig, ss_krig = OK.execute('grid', grid_x, grid_y)

# z_krig is shape (len(grid_y), len(grid_x)); plot in grayscale with axes and title
extent = (grid_x.min(), grid_x.max(), grid_y.min(), grid_y.max())
plt.figure(figsize=(7,7))
plt.imshow(z_krig, origin='lower', extent=extent, cmap='gray')
plt.colorbar(label='EC30.pred (mS/m)')
plt.xlabel('Easting')
plt.ylabel('Northing')
plt.title('Soil Apparent Electrical Conductivity (mS/m) — Ordinary Kriging')
plt.tight_layout()
plt.show()
